# NETWORK CONNECTIONS

For the tracing usecase, cross-border connection of networks is fundamental to the objectives. A logical approach is applied where a network connection object is created, to indicate two nodes that exist in different networks make reference to the same point in the real world object. 

To achieve this, the nodes that make reference to the same real world object, or at least indicate the flow of water from one region or country to another, are identified. 

This entails;
- Extracting the endpoints of one network with no begin points, falling along the border.
- Extracting the beginpoints of another networkfalling along the same border.
Performing an **sjoin_nearest** operation, results will include multiple output records for a single input record where there are multiple equidistant nearest or intersected neighbors.

Because of possible precision errors, a buffer along the border is used to identify the two sets of nodes.

These nodes then form the network connection object, as cross-border connected, cross-border identical or intermodal connections.

This code was developed using the three regions of Belgium, and can be replicated to any regions/countries by declaring the appropriate labels and columns.

In [1]:
import os
import sys
path = os.path.dirname(os.path.abspath(''))
os.chdir(path)
print(path)
sys.path.insert(0, path)

C:\workdir\develop\repository\go-peg


In [3]:
import geopandas as gpd
import pandas as pd

from shapely.geometry import Point, LineString, MultiLineString, MultiPoint, Polygon
from shapely import wkt
from shapely.ops import nearest_points

from src.config import config

### 1. Load datasets

In [4]:
def load_datasets(path, crs):
    """
    Loads the data from the given path, 
    and prints the shape and crs of the data.
    """
    data = gpd.read_file(path)
    data = data.to_crs(crs)
   
    return data


def create_network_connection(dataset1, unique_id1, dataset2, unique_id2, d1_border, d2_border, buffer_length, crs):
    """ This function generates a network connection table, with nodes belonging to two networks
    given as input. The network connection table is created by intersecting the two border polygons,
    and extracting the points that fall within the polygon.

        Using sjoin_nearest, the nodes falling closest to each other along the border 
    are matched to each other. 
    
    To use this function, select a (dataset1), and corresponding region border, 
    which is the dataset to which you match the other (dataset2)."""

    df1 = load_datasets(dataset1, crs) #wal_points
    df2 = load_datasets(dataset2, crs) #vl_points
    dataset1_border = load_datasets(d1_border, crs)
    dataset2_border = load_datasets(d2_border, crs)

    dataset1_buffer = dataset1_border.buffer(buffer_length)
    dataset2_buffer = dataset2_border.buffer(buffer_length)

    #intersection of the two buffers
    buffer_intersection = dataset1_buffer.intersection(dataset2_buffer)

    #extract the point data within the border buffer strip
    df1_points = df1.clip(buffer_intersection)
    df2_points = df2.clip(buffer_intersection)

    network_conn = gpd.sjoin_nearest(df1_points, df2_points)
    print(network_conn.columns)
    
    if unique_id1 == unique_id2:
        network_connection = network_conn[[(unique_id1 + '_left'), (unique_id2 + '_right'), 'geometry']].rename(columns={(unique_id1 + '_left'): 'id_element1', (unique_id2 + '_right'): 'id_element2'})
    else:
        network_connection = network_conn[[unique_id1, unique_id2, 'geometry']].rename(columns={unique_id1: 'id_element1', unique_id2: 'id_element2'})
    
    network_connection_gdf = gpd.GeoDataFrame(network_connection, geometry='geometry').reset_index(drop=True)

    return network_connection_gdf


In [ ]:
#path = config.data_src / "flanders_hydro_network/Wlas.shp"
#path = config.data_dest / "flanders_hydro_network/Wlas.shp"

In [9]:
vl_water = config.data_dest / 'final_vl_nodes_combined.shp'
bxl_water = config.data_dest / 'bxl_waterNodes.shp'
vl_border = config.data_src / 'BE_boundaries/flanders.shp'
bxl_border = config.data_src / 'BE_boundaries/bruxelles.shp'
wal_water = config.data_dest / 'final_wal_nodes_combined.shp'
wal_border = config.data_src / 'BE_boundaries/wallonie.shp'

In [10]:
bxl_vl = create_network_connection(bxl_water, 'node_id', vl_water, 'node_id', bxl_border, vl_border, 50, 'EPSG:31370')

Index(['node_id_left', 'source_left', 'sewernode__left', 'dcpCode', 'dcpName',
       'dcpWaterBo', 'dcpState', 'geometry', 'index_right', 'node_id_right',
       'source_right', 'sewernode__right', 'STATUS', 'LBLTYPE', 'NRKPNT'],
      dtype='object')


In [11]:
bxl_vl = bxl_vl.drop(['geometry'], axis=1)
bxl_vl.head()

,id_element1,id_element2
0,NO25019898,VL248
1,NO25019967_1,VL43992
2,NO25020016,VL26461
3,NO25025490,VL50251
4,NO25020010,VL59880


In [12]:
wal_vl = create_network_connection(wal_water, 'node_id', vl_water, 'node_id', wal_border, vl_border, 50, 'EPSG:31370')

Index(['node_id_left', 'source_left', 'sewernode__left', 'dcpCode', 'dcpName',
       'dcpWaterBo', 'dcpState', 'geometry', 'index_right', 'node_id_right',
       'source_right', 'sewernode__right', 'STATUS', 'LBLTYPE', 'NRKPNT'],
      dtype='object')


In [13]:
wal_vl = wal_vl.drop(['geometry'], axis=1)

In [17]:
#wal_vl.to_csv(r'data_transform\wal_vl_network_connection.csv')
#bxl_vl.to_csv(r'data_transform\bxl_vl_network_connection.csv')